In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

# Loading LangSmith Env Variables 
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_OTEL_ENDPOINT = os.getenv("LANGSMITH_OTEL_ENDPOINT")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")

# Loading AWS Env Variables 

AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION_NAME=os.getenv("AWS_REGION_NAME")

### Part I. Tracing Single Model Invocations

#### Method 1. Tracing with LangChain integration with AWS (Recommended)

To trace Bedrock models, we highly recommend invoking them using LangChain's integration with AWS Bedrock. 

LangChain offers two integrations today to invoke models through [Bedrock](https://python.langchain.com/docs/integrations/chat/bedrock/)
- The first option is **ChatBedrock**, which is utilizing the **invoke_model** function through AWS. https://python.langchain.com/api_reference/aws/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html
- The second option is **ChatBedrockConverse**, which is utilizing the **Converse API** through AWS. https://python.langchain.com/api_reference/aws/chat_models/langchain_aws.chat_models.bedrock_converse.ChatBedrockConverse.html

This is a native way to trace single invocations of LLM call to LangSmith

In [8]:
%pip install -qU langchain-aws

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_aws import ChatBedrock

# Initialize model 
llm = ChatBedrock(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
    region_name=AWS_REGION_NAME,
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

[Example trace](https://smith.langchain.com/public/f1883172-abba-415b-9e25-31c0fbca1f63/r) from invoking the Bedrock model, below

In [7]:
# Invoke LLM model
llm.invoke([("human", "I love programming.")])

AIMessage(content="That's great to hear! Programming is such a rewarding skill. What aspects of programming do you enjoy most? Are you drawn to solving complex problems, building applications, or perhaps the creative freedom it offers? If you're working on any particular projects or learning a specific language right now, I'd be happy to discuss that further.", additional_kwargs={'usage': {'prompt_tokens': 11, 'completion_tokens': 70, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 81}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}, response_metadata={'usage': {'prompt_tokens': 11, 'completion_tokens': 70, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 81}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'us.anthropic.claude-3-7-sonnet-2025021

#### Method 2. Tracing with Traceable decorator

In [33]:
import json
from langsmith import traceable

# Extract only messages as input 
def process_inputs(inputs):
    try:
        body = inputs.get("body", "{}")
        parsed_body = json.loads(body)
        return {"messages" : parsed_body["messages"]}
    except Exception as e:
        return inputs 

# Defining a function to invoke model, trace the function call to LangSmith using Traceable decorator
@traceable(run_type="llm", process_inputs = process_inputs)
def run_model(client, modelId, contentType, accept, body): 
     # Invoke model 
    response = client.invoke_model(modelId=modelId,contentType=contentType,accept=accept, body=body)
    return json.loads(response.get("body").read())

In [34]:
import boto3

session = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
    region_name=AWS_REGION_NAME
)
client = session.client("bedrock-runtime")

prompt = "what's the current time in UTC?"

# Invoking the model 
run_model(
    client=client,
    modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps({
        "anthropic_version" : "bedrock-2023-05-31",
        "max_tokens": 1500,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2
    }) 
)

{'id': 'msg_bdrk_01AZVLoRYEHndzYYxqNGiHv7',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-7-sonnet-20250219',
 'content': [{'type': 'text',
   'text': "The current UTC time is approximately 21:13 on May 7, 2024. Please note that I don't have real-time clock access, so this is based on my last update and may be off by a few minutes."}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 15,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 53}}

#### Method 3. Tracing with OTel

Through this method, you won't need to change at all how you are invoking the model, but two things need to be done: 
- First, set up an OTEL tracer, exporter & Instrumenter 
- Second, certain spans would need to be set in order to fully capture the metrics to LangSmith

In [3]:
pip install -qU openinference-instrumentation-bedrock

Note: you may need to restart the kernel to use updated packages.


In [3]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    BatchSpanProcessor,
)
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

## langsmith 
# Create the OTLP exporter
otlp_exporter = OTLPSpanExporter(
    endpoint=LANGSMITH_OTEL_ENDPOINT,
    headers={"x-api-key": LANGSMITH_API_KEY, "Langsmith-Project": LANGSMITH_PROJECT}
)

# Set up the trace provider
provider = TracerProvider()
processor = BatchSpanProcessor(otlp_exporter)
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)


Next, we will invoke the model and add any necessary metrics to the trace

In [5]:
import boto3
import json

session = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
    region_name=AWS_REGION_NAME
)
client = session.client("bedrock-runtime")

prompt = "what's the current time in UTC?"
messages = [{"role": "user", "content": prompt}]

tracer = trace.get_tracer(__name__)
with tracer.start_as_current_span("call_bedrock") as span:
    span.set_attribute("langsmith.span.kind", "LLM")
    
    # Log every input message  
    for i, message in enumerate(messages):
        span.set_attribute(f"gen_ai.prompt.{i}.content", str(message["content"]))
        span.set_attribute(f"gen_ai.prompt.{i}.role", str(message["role"]))

    # Invoke model 
    response = client.invoke_model(
        modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        contentType="application/json",
        accept="application/json",
        body=json.dumps({
            "anthropic_version" : "bedrock-2023-05-31",
            "max_tokens": 1500,
            "messages": messages,
            "temperature": 0.2
        })
    )

    # Loads results 
    response_body = json.loads(response.get("body").read())

    # Set output results and tokens 
    span.set_attribute("gen_ai.request.model", response_body["model"])
    span.set_attribute("gen_ai.completion.0.content", response_body["content"][0]["text"])
    span.set_attribute("gen_ai.completion.0.role", response_body["role"])
    span.set_attribute("gen_ai.usage.prompt_tokens", response_body["usage"]["input_tokens"])
    span.set_attribute("gen_ai.usage.completion_tokens", response_body["usage"]["output_tokens"])

The resulting trace will look like this [example](https://smith.langchain.com/public/d3752822-91cf-4ffb-a58c-b32397e89b31/r)

### Part II. Tracing AWS Bedrock Agents

Similar to the last method, we will be using OpenTelemetry to trace an agent, first setting up an OTEL tracer, exporter & Instrumenter 

Note: TraceProvider and Instrumentor cannot be overriden. So you may need to restart the kernel if you have already ran the above example. 

In [3]:
pip install -qU openinference-instrumentation-bedrock

Note: you may need to restart the kernel to use updated packages.


In [4]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

from openinference.instrumentation.bedrock import BedrockInstrumentor

# Create the OTLP exporter
otlp_exporter = OTLPSpanExporter(
    endpoint=LANGSMITH_OTEL_ENDPOINT,
    headers={"x-api-key": LANGSMITH_API_KEY, "Langsmith-Project": LANGSMITH_PROJECT}
)

# Set up the trace provider
provider = TracerProvider()
processor = BatchSpanProcessor(otlp_exporter)
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)

# Start the instrumentor for Bedrock
BedrockInstrumentor().instrument(tracer_provider=provider)

Now, you can invoke the agent, and a trace [example](https://smith.langchain.com/public/1ff41604-7e09-4e12-aa60-c46c3f81849c/r) will be sent to LangSmith. 

In [5]:
import time 
import boto3
import os
import json

# Create client  
runtime_client = boto3.client(
    "bedrock-agent-runtime", 
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
    region_name=AWS_REGION_NAME
)

# Invoke agent   
timestamp = int(time.time())
response = runtime_client.invoke_agent(
    agentId="A9RFZVS29O",
    agentAliasId="ZZOIZSFUBS",
    inputText="what's the time now in utc?",
    sessionId=f"default-session1-{timestamp}",
    enableTrace=True,
)


# Output Result 
completion = ""
for event in response.get("completion"):
    if "chunk" in event:
        completion += event["chunk"]["bytes"].decode()
    elif "trace" in event:
        print(event["trace"])
print(completion)

{'agentAliasId': 'ZZOIZSFUBS', 'agentId': 'A9RFZVS29O', 'agentVersion': '1', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-2:640174622193:agent-alias/A9RFZVS29O/ZZOIZSFUBS'}], 'eventTime': datetime.datetime(2025, 6, 19, 1, 56, 1, 85660, tzinfo=tzutc()), 'sessionId': 'default-session1-1750298150', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'foundationModel': 'anthropic.claude-3-5-sonnet-20240620-v1:0', 'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are a helpful assistant that answers customer questions.  You have been provided with a set of functions to answer the user\'s question. You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the question and the previous conversations before creating a plan. - ALWAYS optimize the plan by u